In [3]:
from os import path
import pandas as pd
import numpy as np
import importlib

# setting path
import os, sys
current_dir = os.path.abspath('')
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


from source.load_data.wdc.load_wdc_dataset import EnglishDatasetLoader
from source.utils.levenshtein_distance import compute_lev_dist_for_pairs
from source.emb_extr_res.emb_extr_res import get_embeddings_df
from source.utils.visualization import plot_histogram
from source.probing.similarity_pairs_probing.get_data import get_pair_similarity_probing_task_df
from config import TYPE, SIZE, EMBEDDING_PATH_PROBING, DATA_PATH, MODEL_TYPE, EMBEDDING_PATH_RAW
from source.emb_extr_res.emb_extr_res import get_embeddings_df, get_pairs_similarity_df, get_pretrain_agg_similarity
from source.load_data.wdc.load_wdc_dataset import EnglishDatasetLoader
from source.probing.brand_names import prepare_brands_list, brands_in_title_check, prepare_new_dataset, drop_brands
from source.probing.length import prepare_probing_len
from source.probing.words import words_in_title_check
from source.probing.load_data import load_files_probing_tasks


In [ ]:
print("TYPE:", TYPE)
print("SIZE:", SIZE)
print("MODEL:", MODEL_TYPE)

In [ ]:
train_embeddings_path, embedding_train_df, train_df = load_files_probing_tasks(EMBEDDING_PATH_RAW, DATA_PATH, TYPE, SIZE)

# Probing Task: Sentence Length

In [ ]:
train_df_left = train_df[["id_left", "title_left"]]
train_df_right = train_df[["id_right", "title_right"]]
train_df_left = train_df_left.drop_duplicates().rename({"id_left" : "id", "title_left" : "title"}, axis = 'columns')
train_df_right = train_df_right.drop_duplicates().rename({"id_right" : "id", "title_right" : "title"}, axis = 'columns')

df_train_all = pd.concat([train_df_right, train_df_left])

# dataframe with only titles 
df_train_titles = df_train_all.groupby("id").first().reset_index()

# prepare dataframe with embeddings, label, offer id 
_, df_probing_len_sentence = prepare_probing_len(df_train_titles, train_embeddings_path)

df_probing_len_sentence = df_probing_len_sentence.drop(["id"], axis = 1)
df_probing_len_sentence.to_csv(os.path.join(EMBEDDING_PATH_PROBING, 'df_probing_len_sentence.csv'), index = False)